# Weather generator

***
## Weather conversion from hourly to daily


In [232]:
# only for reading the data as dataframe

def read_hly(file):
    indata = pd.read_csv(file, encoding='utf-16', sep='\t', header=1, skipfooter=1, parse_dates=[0], thousands=',',engine='python')
    indata = indata.drop(indata.columns[[-1, -2]],axis = 1)
    indata = indata.sort_values(by=['Time (LST)'])
    indata = indata.interpolate(method='linear', limit_direction='forward', axis=0)
    indata = indata.set_index('Time (LST)')

    
    headlist = indata.columns.values.tolist()
    print(len(headlist), headlist)

    return indata

# Summarize the data into daily from hourly

In [245]:
def hly2dly(indata):
    print(indata.info())

    ws_avg_dly = indata['Wind Speed (mph)'].resample('D').mean()   # wind speed
    ws_avg_dly.rename('AVG_WS (mph)', inplace=True)

    pcpin_sum_dly = indata['Precipitation (in)'].resample('D').sum()   # solar radiation
    pcpin_sum_dly.rename('precip (in)', inplace=True)
    pcpmm_sum_dly = pcpin_sum_dly*25.4  # convert to mm
    pcpmm_sum_dly.rename('precip (mm)', inplace=True)

    rh_avg_dly = indata['Relative Humidity (%)'].resample('D').mean()   # relative humidity
    rh_avg_dly.rename('AVG_RH (%)', inplace=True)

    tf_avg_dly = indata['Air Temp (°F)'].resample('D').mean()   # max temperature
    tf_avg_dly.rename('AVG_Temp (°F)', inplace=True)
    tc_avg_dly = (tf_avg_dly-32)*5/9  # convert to celsius
    tf_avg_dly.rename('AVG_Temp (°C)', inplace=True)


    sr_avg_dly = indata['Solar Radiation (W/m²)'].resample('D').mean()   # solar radiation
    sr_avg_dly.rename('SUM_SR (W/m²)', inplace=True)

    def cal_svp(value):
        # formula for saturation vapor pressure
        # IF(K3>0,0.6108*EXP(17.27*K3/(237.3+K3)),0.6108*EXP(17.27*0.5/(237.3+0.5)))
        if value > 0:
            return np.exp(17.27*value/(237.3+value))*0.6108
        else:
            return np.exp(17.27*0.5/(237.3+0.5))*0.6108
    svp_dly = tc_avg_dly.apply(cal_svp)  # saturation vapor pressure
    svp_dly.rename('Saturation Vapor Pressure (kPa)', inplace=True)

    vp_dly = rh_avg_dly/100*svp_dly*10  # vapor pressure
    vp_dly.rename('Vapor Pressure (mb)', inplace=True)


    dlydata = pd.concat([ws_avg_dly, pcpmm_sum_dly, sr_avg_dly, tc_avg_dly, rh_avg_dly, svp_dly, vp_dly], axis=1)
    dlydata = dlydata.round(3)
    return dlydata



# Below is implementation block

In [246]:
import pandas as pd
import numpy as np

# get today's date as string
from datetime import date
today = date.today().strftime("%Y%m%d")

indata = read_hly('Table (5-min).csv')
indata.head()

dlydata = hly2dly(indata)
dlydata.head()

dlydata.to_csv('Penman_weather_'+today+'_.csv') 

/var/folders/_t/xf80sxmn3c99xrwsspdr3tc00000gp/T/ipykernel_27687/3861858551.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  indata = pd.read_csv(file, encoding='utf-16', sep='\t', header=1, skipfooter=1, parse_dates=[0], thousands=',',engine='python')


24 ['Air Temp (°F)', '0.5 m Air Temp (°F)', '1.5 m Air Temp (°F)', '3 m Air Temp (°F)', 'Relative Humidity (%)', 'Precipitation (in)', 'Accumulated Precip (in)', 'Solar Radiation (W/m²)', 'Wind Speed (mph)', 'Wind Direction (°)', 'Wind Gust (mph)', '4" Bare Soil Temp (°F)', '4" Grass Soil Temp (°F)', '2" Soil Temp (°F)', '2" Soil Water Content (%)', '4" Soil Temp (°F)', '4" Soil Water Content (%)', '8" Soil Temp (°F)', '8" Soil Water Content (%)', '20" Soil Temp (°F)', '20" Soil Water Content (%)', 'Inversion Strength', 'Max Inversion', 'Battery Voltage']
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8634 entries, 2024-04-14 15:35:00 to 2024-05-14 15:00:00
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Air Temp (°F)               8634 non-null   float64
 1   0.5 m Air Temp (°F)         8634 non-null   float64
 2   1.5 m Air Temp (°F)         8634 non-null   float64
 3   3 m A